In [26]:
# # Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from Dataset_OnlyOneHotEncodingApplied import *
# # Set options
# pd.options.display.max_rows = 999
# pd.options.display.max_columns = 999

# train_x_raw = pd.read_csv("../01-Data/X_train.csv", low_memory = True, index_col=0)
# train_y_raw = pd.read_csv("../01-Data/y_train.csv", low_memory = True, index_col=0)
# test_x_raw = pd.read_csv("../01-Data/X_test.csv", low_memory=True, index_col=0)

# df_train = pd.DataFrame(train_x_raw)
# df_test = pd.DataFrame(test_x_raw)
# df_y = pd.DataFrame(train_y_raw)

# Simple Model Run
##  xgBoost model set up

In [44]:
from sklearn.metrics import log_loss

label_mapping = {-1: 0, 1: 1, 2: 2, 3: 3, 4: 4}
df_y = df_y.replace(label_mapping)


# dtrain = xgb.DMatrix(df_train, label=df_y, enable_categorical=True)
# dtest = xgb.DMatrix(df_test, enable_categorical=True)


import cupy as cp

# Convert features and labels to Cupy arrays
df_train_gpu = cp.asnumpy(df_train)
df_y_gpu = cp.asnumpy(df_y)

# Create DMatrix from Cupy arrays
dtrain = xgb.DMatrix(df_train_gpu, label=df_y_gpu, enable_categorical=True)

# Convert test data to Cupy arrays
df_test_gpu = cp.asnumpy(df_test)

# Create DMatrix for test data
dtest = xgb.DMatrix(df_test_gpu, enable_categorical=True)


params = {
    'max_depth': 8,
    'eta': 0.1,
    'objective': 'multi:softprob',
    'num_class': 5,
    'eval_metric': 'mlogloss',
    'device' : 'cuda',
    'predictor': 'gpu_predictor'
}



num_boost_round = 1000

## Xgboost Model GridSearchCV

In [46]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

# Assume df_train, df_test, and df_y are already defined and preprocessed
# as per your setup

# Define the parameter grid to search
param_grid = {
    'device' : ['cuda'],
    'max_depth': [8],
    'learning_rate': [0.05, 0.1],
    'n_estimators': [500, 1000],
    'subsample': [0,7, 0.8],
    'gamma': [0.1, 0.3],
    'colsample_bytree': [0.7, 0.8],
}

# Instantiate the XGBClassifier (note: use objective 'multi:softprob' for multi-class)
xgb_model = XGBClassifier(objective='multi:softprob', num_class=5, eval_metric='mlogloss', use_label_encoder=False,device='cuda')

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='neg_log_loss', n_jobs=-1, cv=3, verbose=2)

# Fit the GridSearchCV object to find the best parameters
grid_search.fit(df_train_gpu, df_y_gpu)

# Best parameter set found
print("Best parameters found: ", grid_search.best_params_)

# Best score
print("Best score (neg_log_loss): ", grid_search.best_score_)

# You can also use the best estimator directly to make predictions
# best_estimator = grid_search.best_estimator


Fitting 3 folds for each of 48 candidates, totalling 144 fits


[CV] END colsample_bytree=0.7, device=cuda, gamma=0.1, learning_rate=0.05, max_depth=8, n_estimators=500, subsample=7; total time=   2.0s
[CV] END colsample_bytree=0.7, device=cuda, gamma=0.1, learning_rate=0.05, max_depth=8, n_estimators=500, subsample=7; total time=   2.1s
[CV] END colsample_bytree=0.7, device=cuda, gamma=0.1, learning_rate=0.05, max_depth=8, n_estimators=1000, subsample=7; total time=   2.4s
[CV] END colsample_bytree=0.7, device=cuda, gamma=0.1, learning_rate=0.05, max_depth=8, n_estimators=500, subsample=7; total time=   2.5s
[CV] END colsample_bytree=0.7, device=cuda, gamma=0.1, learning_rate=0.05, max_depth=8, n_estimators=1000, subsample=7; total time=   3.8s
[CV] END colsample_bytree=0.7, device=cuda, gamma=0.1, learning_rate=0.05, max_depth=8, n_estimators=1000, subsample=7; total time=   3.8s
[CV] END colsample_bytree=0.7, device=cuda, gamma=0.1, learning_rate=0.1, max_depth=8, n_estimators=500, subsample=7; total time=   2.9s
[CV] END colsample_bytree=0.7, d

/home/jeonghan/.local/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:21:17] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=0.7, device=cuda, gamma=0.1, learning_rate=0.1, max_depth=8, n_estimators=500, subsample=0; total time= 2.1min


/home/jeonghan/.local/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:21:18] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
/home/jeonghan/.local/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:21:18] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- 

...

[CV] END colsample_bytree=0.7, device=cuda, gamma=0.3, learning_rate=0.1, max_depth=8, n_estimators=1000, subsample=0; total time= 4.4min


/home/jeonghan/.local/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:35:17] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=0.7, device=cuda, gamma=0.3, learning_rate=0.1, max_depth=8, n_estimators=1000, subsample=0; total time= 4.4min
[CV] END colsample_bytree=0.7, device=cuda, gamma=0.3, learning_rate=0.05, max_depth=8, n_estimators=1000, subsample=0.8; total time= 9.3min
[CV] END colsample_bytree=0.7, device=cuda, gamma=0.3, learning_rate=0.05, max_depth=8, n_estimators=1000, subsample=0.8; total time= 9.3min
[CV] END colsample_bytree=0.7, device=cuda, gamma=0.3, learning_rate=0.05, max_depth=8, n_estimators=1000, subsample=0.8; total time= 9.3min
[CV] END colsample_bytree=0.8, device=cuda, gamma=0.1, learning_rate=0.1, max_depth=8, n_estimators=500, subsample=7; total time=   1.8s
[CV] END colsample_bytree=0.8, device=cuda, gamma=0.1, learning_rate=0.1, max_depth=8, n_estimators=500, subsample=7; total time=   1.0s
[CV] END colsample_bytree=0.8, device=cuda, gamma=0.1, learning_rate=0.1, max_depth=8, n_estimators=500, subsample=7; total time=   1.3s
[CV] END colsample_bytree=0.

/home/jeonghan/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
48 fits failed out of a total of 144.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
48 fits failed with the following error:
Traceback (most recent call last):
  File "/home/jeonghan/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/jeonghan/.local/lib/python3.10/site-packages/xgboost/core.py", line 730, in inner_f
    return func(**kwargs)
  File "/home/jeonghan/.local/lib/python3.10/site-packages/xgboost/sklearn.py", line 1519, in fit
    self._Booster = train(
  File "/home/jeonghan/.local/lib/python3.10

Best parameters found:  {'colsample_bytree': 0.7, 'device': 'cuda', 'gamma': 0.3, 'learning_rate': 0.05, 'max_depth': 8, 'n_estimators': 500, 'subsample': 0.8}
Best score (neg_log_loss):  -0.855990098390015


## Cross Validation

In [ ]:
from xgboost import cv

# params and num_boost_round provided above
xgb_cv = cv(dtrain=dtrain, params=params, nfold=10,
            num_boost_round=num_boost_round, early_stopping_rounds=5,
            metrics="mlogloss", as_pandas=True, seed=123)

xgb_cv

/home/jeonghan/.local/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [14:06:54] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
/home/jeonghan/.local/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [14:06:55] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoostError: [14:06:55] /workspace/src/metric/multiclass_metric.cu:35: Check failed: label_error >= 0 && label_error < static_cast<int32_t>(n_class): MultiClassEvaluation: label must be in [0, num_class), num_class=1 but found 3 in label
Stack trace:
  [bt] (0) /home/jeonghan/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x1ba24e) [0x7fcef228224e]
  [bt] (1) /home/jeonghan/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x9a5666) [0x7fcef2a6d666]
  [bt] (2) /home/jeonghan/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x4efea5) [0x7fcef25b7ea5]
  [bt] (3) /home/jeonghan/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x4f0168) [0x7fcef25b8168]
  [bt] (4) /home/jeonghan/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x4c8ab0) [0x7fcef2590ab0]
  [bt] (5) /home/jeonghan/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(XGBoosterEvalOneIter+0x2f6) [0x7fcef2232bb6]
  [bt] (6) /lib/x86_64-linux-gnu/li

## Xgboost Training

In [ ]:
evals_result = {}
bst = xgb.train(params, dtrain, num_boost_round, 
                evals=[(dtrain, 'train')], evals_result=evals_result, 
                verbose_eval=False)
print(f"Training Multiclass Logarithmic Loss: {evals_result['train']['mlogloss'][-1]}")

y_test_probs = bst.predict(dtest)

class_order = [0, 1, 2, 3, 4]
class_mapping = {class_label: f"Class_{class_label}" for class_label in class_order}

y_train_probs = bst.predict(dtrain)
val_log_loss = log_loss(df_y, y_train_probs, labels=class_order)
print(f"Validation Multiclass Logarithmic Loss: {val_log_loss}")

Training Multiclass Logarithmic Loss: 0.7841006598490445
Validation Multiclass Logarithmic Loss: 0.7841006488289419


## Generate Submission csv

In [ ]:
submission_df = pd.DataFrame(y_test_probs, columns=class_mapping.values())
submission_df.columns = ['no answer', 'very important', 'quite important', 'not important', 'not at all important']
submission_df.insert(0, 'id', df_test.index)

# Save the submission file
# submission_file = ('submission.csv')
# submission_df.to_csv(submission_file, index=False)